<a href="https://colab.research.google.com/github/Uday170604/CSRBOX_IBMSKILLBUILD/blob/main/Health_chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -qU \
  google-generativeai==0.8.5 \
  google-ai-generativelanguage==0.6.15 \
  langgraph \
  langchain \
  langchain-google-genai \
  openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.8/143.8 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 757.5/757.5 kB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.5/216.5 kB 14.0 MB/s eta 0:00:00


In [ ]:
import os
import getpass
from langgraph.graph import StateGraph, END
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage

In [ ]:
os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter your gemini key")

Enter your gemini key··········


In [ ]:
llm = ChatGoogleGenerativeAI(model="models/gemini-1.5-flash-latest", temperature=0.2)

In [ ]:
def get_problem(state: dict) -> dict:
  problem = input("This is an Expert A.I guide for your health problem , Please enter the problem to be solved")
  state["problem"] = problem
  return state

In [ ]:
def classify_problem(state: dict) -> dict:
    prompt = (
        "You are a helpful health doctor specialist. Classify the user's health concern below into one of the categories:\n"
        "- Awareness\n- Action\n- Emergency\n\n"
        f"problem: {state['problem']}\n"
        "Respond with only one word: Awareness, Action, or Emergency.\n"
        "#Example: input: I’m experiencing chest pain and shortness of breath. → output: Emergency"
    )
    response = llm.invoke([HumanMessage(content=prompt)])
    category = response.content.strip()
    print(f"LLM Classifies the problem as: {category}")
    state["category"] = category


In [ ]:
def problem_router(state: dict) -> dict:
    """
    Routes a health concern based on its classified category.
    Categories:
    - 'awareness'  → General health education or non-urgent knowledge
    - 'action'     → Needs a consultation, check-up, or lifestyle change
    - 'emergency'  → Urgent medical issue requiring immediate attention
    """
    cat = state["category"].lower()
    if "awareness" in cat:
        return "awareness"
    elif "emergency" in cat:
        return "emergency"
    elif "action" in cat:
        return "action"
    else:
        return "awareness"


In [ ]:
def awareness_node(state: dict) -> dict:
    prompt = (
        f"You are a knowledgeable health educator.\n"
        f"User's concern: {state['problem']}\n"
        "Give an informative, clear explanation to raise the user's awareness about this health-related issue."
    )
    response = llm.invoke([HumanMessage(content=prompt)])
    state["answer"] = response.content.strip()
    return state

def emergency_node(state: dict) -> dict:
    prompt = (
        f"You are a medical emergency specialist.\n"
        f"Emergency situation: {state['problem']}\n"
        "Provide urgent medical advice or immediate steps the user should take to ensure their safety."
    )
    response = llm.invoke([HumanMessage(content=prompt)])
    state["answer"] = response.content.strip()
    return state

def action_node(state: dict) -> dict:
    prompt = (
        f"You are a helpful health doctor specialist.\n"
        f"User's concern: {state['problem']}\n"
        "Please provide a clear, specific action the user can take to address this health issue (e.g., lifestyle change, consultation, first aid)."
    )
    response = llm.invoke([HumanMessage(content=prompt)])
    state["answer"] = response.content.strip()
    return state


In [ ]:
builder = StateGraph(dict)

builder.set_entry_point("get_problem")

builder.add_node("get_problem", get_problem)
builder.add_node("classify", classify_problem)
builder.add_node("awareness", awareness_node)
builder.add_node("emergency", emergency_node)
builder.add_node("action", action_node)

builder.add_edge("get_problem", "classify")

builder.add_conditional_edges("classify", problem_router, {
    "awareness": "awareness",
    "emergency": "emergency",
    "action": "action"
})

builder.add_edge("awareness", END)
builder.add_edge("emergency", END)
builder.add_edge("action", END)


In [ ]:
graph = builder.compile()

final_state = graph.invoke({})

print("Final Output:")
print(final_state["answer"])

This is an Expert A.I guide for your health problem , Please enter the problem to be solvedi have fever
LLM Classifies the problem as: Awareness
Final Output:
Having a fever means your body temperature is higher than normal.  A normal body temperature is generally considered to be around 98.6°F (37°C), but it can fluctuate slightly throughout the day and between individuals.  A fever is usually defined as a temperature of 100.4°F (38°C) or higher, measured orally.  Rectal temperatures are generally a degree higher, and axillary (armpit) temperatures are usually a degree lower.

**What causes a fever?**

Fever isn't an illness itself, but rather a *symptom* of an underlying condition.  Your body raises its temperature as a defense mechanism to fight off infection, such as:

* **Viral infections:**  The common cold, flu, COVID-19, and other viral illnesses are common causes.
* **Bacterial infections:**  These can range from strep throat to pneumonia to urinary tract infections.
* **Other